# Create a table with TCGA data

In [ ]:
%load_ext watermark
%watermark -v -m  -u -n -p pandas,numpy,matplotlib,regex -a Filippo_Valle -g -r -b -w

In [ ]:
# import libraries
import os
import sys
import pandas as pd
import numpy as np
import regex as re
from matplotlib import pyplot as plt
import time

Set *working_dir* to the directory where you downloaded files in *data*

In [ ]:
working_dir = "/home/jovyan/work/phd/cnv/"
os.chdir(working_dir)
dirs = os.listdir("data")

In [ ]:
len(dirs)

Prepare a list of genes that satisfied filters described by *Dey et al.* [Visualizing the structure of RNA-seq expression data using grade of membership models](https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1006599)

In [ ]:
#df_genes = pd.read_csv("https://stephenslab.github.io/count-clustering/project/utilities/gene_names_all_gtex.txt", header=None).set_index(0)
df_genes = pd.read_csv("miRNA.txt").set_index("Gene stable ID").drop("miRBase ID",1)
select_genes = df_genes.index.values

In [ ]:
df = df_genes.copy()
#print(len(df['gene']))
#df = df.loc[select_genes,:]
df.head(10)

In [ ]:
len(df)

In [ ]:
##cnv
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("TCGA-[A-Z]*.[a-z0-9]{8}-[a-z0-9]{4}-[a-z0-9]{4}-[a-z0-9]{4}-[a-z0-9]{12}.gene_level_copy_number.tsv",element):
            cfile = element
            print(element)
            return cfile

In [ ]:
maxacceptables = 15000
added = len(df.columns)
for i,cdirectory in enumerate(dirs):
    if re.match("manifest\.txt",cdirectory):
        print("SKIPPING %s "%cdirectory)
        continue
    if "Icon" in cdirectory:
        print("SKIPPING %s "%cdirectory)
        continue
    if ".DS_Store" in cdirectory:
        print("SKIPPING %s "%cdirectory)
        continue
    cfile = getFilenameFromDir("data/%s"%cdirectory)
    cdf = pd.read_csv(("data/%s/%s"%(cdirectory,cfile)), sep='\t', header=0)
    cdf["gene_id"] = [gene[:15] for gene in cdf['gene_id']]
    cdf.set_index('gene_id',inplace=True)
    cdf = cdf.reindex(index=df_genes.index)
    old_L = len(df.columns)
    df.insert(0,cfile,cdf["copy_number"].values)
    if len(df.columns) != old_L+1:
        print(*sys.exc_info())
        raise(Exception("Not able to add: %s"%cfile))
    if added >= maxacceptables:
        break
print(added, i)

In [ ]:
#df = pd.read_csv(("%s/mainTable.csv"%working_dir))

In [ ]:
print(("genes:%d\trealizations:%d"%(len(df.index),len(df.columns))))

In [ ]:
df.astype("Int64")

In [ ]:
df.dropna(how='all', axis=0).astype("Int64").to_csv("mainTable_cnv.csv", index=True)

In [ ]:
df_files = pd.read_csv("files_cnv.dat", index_col=0).reindex(index=df.columns)
df.columns = df_files["cases.0.submitter_id"].values
df.dropna(how='all', axis=0).astype("Int64").to_csv("mainTable_cnv.csv", index=True, header=True)

In [ ]:
df